In [7]:
import hashlib
from os import urandom
from base64 import b64encode, b64decode
from itertools import zip_longest

# Parameters to PBKDF2. Only affect new passwords.
SALT_LENGTH = 12
KEY_LENGTH = 24
HASH_FUNCTION = 'sha256'  # Must be in hashlib.
COST_FACTOR = 10000

def pbkdf2_bin(password, salt, iterations, dklen, hash_func):
    """PBKDF2 using hashlib directly"""
    return hashlib.pbkdf2_hmac(hash_func, password, salt.encode(), iterations, dklen)

def make_hash(password):
    """Generate a random salt and return a new hash for the password."""
    if isinstance(password, str):  # Python 3 uses 'str' for Unicode text
        password = password.encode('utf-8')
    salt = b64encode(urandom(SALT_LENGTH)).decode('utf-8')
    hash_bytes = pbkdf2_bin(password, salt, COST_FACTOR, KEY_LENGTH, HASH_FUNCTION)
    return 'PBKDF2${}${}${}${}'.format(
        HASH_FUNCTION,
        COST_FACTOR,
        salt,
        b64encode(hash_bytes).decode('utf-8'))

def check_hash(password, hash_):
    """Check a password against an existing hash."""
    if isinstance(password, str):  # Python 3 uses 'str' for Unicode text
        password = password.encode('utf-8')
    algorithm, hash_function, cost_factor, salt, hash_a = hash_.split('$')
    assert algorithm == 'PBKDF2'
    hash_a = b64decode(hash_a)
    hash_b = pbkdf2_bin(password, salt, int(cost_factor), len(hash_a), hash_function)
    
    # Constant time comparison
    diff = 0
    for char_a, char_b in zip_longest(hash_a, hash_b):
        diff |= ord(char_a) ^ ord(char_b)
    return diff == 0

# hash_generator.py

# Código do hash_generator já fornecido acima...

if __name__ == "__main__":
    password = input("Digite a senha para gerar o hash: ")
    print(f"Senha digitada: {password}")
    hash_gerado = make_hash(password)
    print(f"Hash gerado: {hash_gerado}")


Senha digitada: connectorpass
Hash gerado: PBKDF2$sha256$10000$OKMrtRlbBmITT+eY$gYWnNI42ymiCbZsG8Lbsj0QXWVkWcxQE


Senha digitada: teste
Hash gerado: PBKDF2$sha256$10000$WeleOC9X8YQFpSO/$rQ2n4Zt6mdneOcSvFf0Nrnfv2iGqVKid

Senha digitada: connectorpass
Hash gerado: PBKDF2$sha256$10000$OKMrtRlbBmITT+eY$gYWnNI42ymiCbZsG8Lbsj0QXWVkWcxQE